## Entities

We identified six basic entities in our schema:

1\. Hospital

2\. Location

3\. Hospital Quality

4\. Weekly Report

5\. Bed Report

6\. Covid Report

We chose these entities to effectively separate information like hospital details and location from weekly reports and quality ratings. This approach reduces redundancy by making sure that each piece of information is stored only once.

## 1\. Hospital

Table for hospital information:

<span style="color: var(--vscode-foreground);">This table represents the 'Hospital' entity and stores information about each hospital.&nbsp;</span> <span style="color: var(--vscode-foreground);">We use 'hospital_pk' as the primary key to uniquely identify each hospital.&nbsp;</span> <span style="color: var(--vscode-foreground);">The 'location_id' is a foreign key referencing the 'location' table, linking each hospital to its location.</span>

In [23]:
CREATE TABLE hospital (
    hospital_pk TEXT PRIMARY KEY,
    hospital_name TEXT NOT NULL,
    location_id INTEGER REFERENCES location(id) ON DELETE SET NULL,
    type_of_hospital TEXT,
    ownership TEXT,
    provides_emergency_services BOOLEAN
);

CREATE TABLE

Total execution time: 00:00:00.029

## 2\. Location

Table to store unique location information:

 The 'Location' entity stores unique geographical information to avoid redundancy when multiple hospitals <span style="color: var(--vscode-foreground);">share the same location details, which helps to minimize duplication.</span>

In [22]:
CREATE TABLE location (
    id SERIAL PRIMARY KEY,
    city TEXT,
    state CHAR(2) NOT NULL,
    zip_code CHAR(5),
    latitude NUMERIC(9, 6),
    longitude NUMERIC(9, 6),
    fips_code VARCHAR(10)
);

CREATE TABLE

Total execution time: 00:00:00.024

## 3\. Hospital Quality

Table to store the quality ratings of each hospital:

<span style="color: var(--vscode-foreground);">This table keeps track of quality ratings for each hospital over time, where</span> <span style="color: var(--vscode-foreground);">'facility_id' links back to the 'hospital' table.&nbsp;</span>   <span style="color: var(--vscode-foreground);">We can see how a hospital's rating changes by date through this table.</span>

In [24]:
CREATE TABLE hospital_quality (
    id SERIAL PRIMARY KEY,
    facility_id TEXT REFERENCES hospital(hospital_pk) ON DELETE CASCADE,
    quality_rating INTEGER CHECK (quality_rating BETWEEN 1 AND 10), 
    rating_date DATE NOT NULL                                   
);

CREATE TABLE

Total execution time: 00:00:00.035

## 4\. Weekly Report

Table to store weekly data reports for each hospital:

This table stores weekly report dates for each hospital, where 'hospital\_weekly\_id' links to the 'hospital' table. We ensure there's only one report per hospital each week.

In [25]:
CREATE TABLE weekly_report (
    id SERIAL PRIMARY KEY,
    hospital_weekly_id TEXT REFERENCES hospital(hospital_pk) ON DELETE CASCADE,
    collection_week DATE NOT NULL,                                -- Week of data collection
    UNIQUE(hospital_weekly_id, collection_week)                   -- Ensure one entry per hospital per week
);

CREATE TABLE

Total execution time: 00:00:00.033

## 5\. Bed Report

Table to store bed-related information:

<span style="color: var(--vscode-foreground);">This table holds data about bed usage for each weekly report.</span> <span style="color: var(--vscode-foreground);">It includes the type of bed and averages of beds available and occupied, where</span> <span style="color: var(--vscode-foreground);">'weekly_bed_report_id' links to the 'weekly_report' table.</span>

In [27]:
CREATE TABLE bed_report (
    id SERIAL PRIMARY KEY,
    weekly_bed_report_id INTEGER REFERENCES weekly_report(id) ON DELETE CASCADE,
    bed_type TEXT NOT NULL,                                       -- Type of bed (e.g., adult, pediatric, ICU)
    beds_available_7_day_avg NUMERIC(10, 2),                      -- Avg. beds available
    beds_occupied_7_day_avg NUMERIC(10, 2),                       -- Avg. beds occupied
    UNIQUE(weekly_bed_report_id, bed_type)                            -- Ensure unique entry per bed type per report
);

CREATE TABLE

Total execution time: 00:00:00.041

## 6\. Covid Report

Table to store Covid-related information:

This table contains data about the number of Covid patients for each weekly report. It includes the average total number of Covid patients and adult ICU Covid patients, and 'weekly\_covid\_id' links to the 'weekly\_report' table.

In [28]:
CREATE TABLE covid_report (
    id SERIAL PRIMARY KEY,
    weekly_covid_id INTEGER REFERENCES weekly_report(id) ON DELETE CASCADE,
    total_covid_7_day_avg NUMERIC (10, 2),                         -- Total COVID patients
    adult_icu_covid_7_day_avg NUMERIC (10, 2)                      -- Adult ICU COVID patients
);

CREATE TABLE

Total execution time: 00:00:00.025

## Summarizing relationships:

| Table | Primary Key | Foreign Key | Relationship |
| --- | --- | --- | --- |
| `location` | `id` | None | Standalone table for unique locations |
| `hospital` | `hospital_pk` | `location_id` references `location(id)` | Links each hospital to a unique location |
| `hospital_quality` | `id` | `facility_id` references `hospital(hospital_pk)` | Links each quality rating to a hospital |
| `weekly_report` | `id` | `hospital_weekly_id` references `hospital(hospital_pk)` | Links each weekly report to a hospital |
| `bed_report` | `id` | `weekly_bed_report_id` references `weekly_report(id)` | Links each bed report to a specific weekly report. Each bed report entry represents details for a specific bed type within that weekly report.<br> |
| `covid_report` | `id` | `weekly_covid_id` references `weekly_report(id)` | Links each covid report to a specific weekly report |

<span style="color: var(--vscode-foreground);">We designed our schema this way to separate data into different tables to keep things organized and avoid duplicates:</span>

<span style="color: var(--vscode-foreground);">- The 'hospital' table stores each hospital's info once.</span>

\- The 'location' table holds location details that can be shared by multiple hospitals.

\- The 'hospital\_quality' table tracks quality ratings over time without repeating hospital info.

\- The 'weekly\_report', 'bed\_report', and 'covid\_report' tables store time-based patient data linked to hospitals.

<span style="color: var(--vscode-foreground);">By linking tables with foreign keys, we connect related data without storing the same information multiple times.</span> <span style="color: var(--vscode-foreground);">This makes our database efficient and easier to maintain.</span>